# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [1]:
import pandas as pd
import sqlite3 
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

In [2]:
#Your code here; use a subquery. No join will be necessary.
cur.execute("""
    SELECT customerNumber, contactLastName, contactFirstName
    FROM customers
    WHERE customerNumber IN (SELECT customerNumber FROM orders WHERE orderDate = '2003-01-31');""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [3]:
cur.execute("""
    SELECT productCode, productName, COUNT(orderNumber) as num_orders
    FROM products
    JOIN orderDetails
    USING(productCode)
    GROUP BY productCode;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,productCode,productName,num_orders
0,S10_1678,1969 Harley Davidson Ultimate Chopper,28
1,S10_1949,1952 Alpine Renault 1300,28
2,S10_2016,1996 Moto Guzzi 1100i,28
3,S10_4698,2003 Harley-Davidson Eagle Drag Bike,28
4,S10_4757,1972 Alfa Romeo GTA,28
...,...,...,...
104,S700_3505,The Titanic,27
105,S700_3962,The Queen Mary,27
106,S700_4002,American Airlines: MD-11S,28
107,S72_1253,Boeing X-32A JSF,28


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [4]:
#Your code here:
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
cur.execute("""
    SELECT DISTINCT productCode, productName, COUNT(customerNumber) as num_customers_ordered
    FROM products
    JOIN orderDetails
    USING(productCode)
    JOIN orders
    USING(orderNumber)
    GROUP BY productCode;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,productCode,productName,num_customers_ordered
0,S10_1678,1969 Harley Davidson Ultimate Chopper,28
1,S10_1949,1952 Alpine Renault 1300,28
2,S10_2016,1996 Moto Guzzi 1100i,28
3,S10_4698,2003 Harley-Davidson Eagle Drag Bike,28
4,S10_4757,1972 Alfa Romeo GTA,28
...,...,...,...
104,S700_3505,The Titanic,27
105,S700_3962,The Queen Mary,27
106,S700_4002,American Airlines: MD-11S,28
107,S72_1253,Boeing X-32A JSF,28


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products Which Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [5]:
#Your code here
cur.execute("""
    SELECT DISTINCT employeeNumber, lastName, firstName, officeCode, offices.city AS office_city, COUNT(customerNumber) as num_customers_ordered
    FROM employees
    JOIN offices USING(officeCode)
    JOIN customers ON employees.employeeNumber = customers.salesRepEmployeeNumber
    JOIN orders USING(customerNumber)
    GROUP BY employeeNumber
    HAVING num_customers_ordered < 20;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,employeeNumber,lastName,firstName,officeCode,office_city,num_customers_ordered
0,1166,Thompson,Leslie,1,San Francisco,14
1,1188,Firrelli,Julie,2,Boston,14
2,1216,Patterson,Steve,2,Boston,18
3,1286,Tseng,Foon Yue,3,NYC,17
4,1611,Fixter,Andy,6,Sydney,19
5,1612,Marsh,Peter,6,Sydney,19
6,1621,Nishi,Mami,5,Tokyo,16
7,1702,Gerard,Martin,4,Paris,12


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit of Over 15K

In [6]:
#Your code here
cur.execute("""
    SELECT DISTINCT employeeNumber, lastName, firstName, COUNT(customerNumber) AS num_customers_with_avg_cred_limit_gt_15k, AVG(creditLimit) AS avg_cred_lim
    FROM employees
    JOIN customers ON employees.employeeNumber = customers.salesRepEmployeeNumber
    GROUP BY employeeNumber
    HAVING avg_cred_lim > 15000
    ORDER BY avg_cred_lim ASC;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,employeeNumber,lastName,firstName,num_customers_with_avg_cred_limit_gt_15k,avg_cred_lim
0,1166,Thompson,Leslie,6,65266.666667
1,1286,Tseng,Foon Yue,7,66614.285714
2,1702,Gerard,Martin,6,71916.666667
3,1188,Firrelli,Julie,6,73916.666667
4,1323,Vanauf,George,8,80887.500000
5,1401,Castillo,Pamela,10,81340.000000
6,1216,Patterson,Steve,6,81533.333333
7,1621,Nishi,Mami,5,83900.000000
8,1504,Jones,Barry,9,85511.111111
9,1611,Fixter,Andy,5,86060.000000


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!